In [1]:
import pygame as pg
from pygame.locals import *
import cv2 as cv
import numpy as np
import sys
from matplotlib import pyplot as plt

import visio as v

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [1]:
import pygame as pg
from pygame.locals import *
import cv2 as cv
import numpy as np
import sys
from matplotlib import pyplot as plt

import visio as v

FPS = 30



# [pygame] Inicialitzem el modul pygame 
pg.init()
# [pygame] Definim les característiques de la pantalla de joc
pg.display.set_caption("[Modul Control] Joc de domino en PyGame")
screen = pg.display.set_mode([800,600])
# [pygame] Registrem el primer frame per simular el video en directe
pg.image.save(screen,'lastFrame.jpg')

'''
fitxaImg = pg.image.load('src/fitxa_66.png')
midaFitxes = (35,70)
'''

game_rotation = None


class Fitxa(pg.sprite.Sprite):
    def __init__(self,src,x,y,scale):
        pg.sprite.Sprite.__init__(self)
        self.src = 'src/fitxes/fitxa_{}.png'.format(src)
        self.scale = scale
        self.original_frame = pg.image.load(self.src)
        self.image = self.original_frame.copy().convert_alpha()
        #self.image.fill((255,127,0))
        self.image = pg.transform.scale(self.image,self.scale)
        self.rect = self.image.get_rect()
        self.rect.center = (self.rect.width/2,self.rect.height/2)
        self.rect.x = x
        self.rect.y = y
        
        # Propietats fitxa dins el joc
        self.drag = False
        self.placed=False
        self.angle = 0.0
        self.perpendicular = False
        
    def __str__(self):
        ret = str("{} :: {} ::{}".format(self.rect.center,self.drag,self.placed))
        return ret
    
    
    def hovered(self,x,y):
        if x in range(self.rect.left, self.rect.right) and y in range(self.rect.top, self.rect.bottom):
            return True
        else:
            return False
    def moveTo(self,x,y):
        self.rect.center = (x,y)
        
    def rotate(self, angle):
        self.angle = (self.angle+angle)%360
        print(self.angle)
        old_center = self.rect.center
        self.image = self.original_frame.copy().convert_alpha()
        
        self.image = pg.transform.scale(self.image,self.scale)
        self.image = pg.transform.rotate(self.image,self.angle)
        self.rect = self.image.get_rect()
        self.rect.center = old_center
        
        
        

        
fitxes = pg.sprite.Group()
fitxes.add(Fitxa(np.random.randint(2,25),np.random.randint(40,640)-35,np.random.randint(75,480)-70,(35,70)))


spriteIndex = 0

# Bucle del joc
running = True
while running:
    
    mx,my = pg.mouse.get_pos()
    
    # [pygame] Pintem el fons 

    # [openCV] Llegim la imatge de la partida
    frame = cv.imread('lastFrame.jpg')
    
    d = v.processarFrame(frame)

    
    
    cv.imshow("[Modul Visio] Estat de la partida processat amb OpenCV",v.mostrarResultat(frame,d))
    
    
    #pg.time.wait(10)
    # [pygame i openCV] Detectem events
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running=False
        if event.type == MOUSEMOTION:            
            for s in fitxes.sprites():
                if s.drag:
                    s.moveTo(mx,my)
                    break
                    
        if event.type == MOUSEBUTTONDOWN:
            if pg.mouse.get_pressed()[0]:
                for s in fitxes.sprites():
                    if s.rect.collidepoint(mx,my):
                        if s.drag:
                            s.drag = False
                            if game_rotation is None:
                                game_rotation = s.angle
                        else:
                            s.drag = True
                        break
            elif pg.mouse.get_pressed()[2]:
                for s in fitxes.sprites():
                    if s.drag:
                        # TODO
                        #ANGLES DEPENENT DEL ESTAT DE LA PARTIDA
                        if game_rotation is None:
                            s.rotate(3.1415)                            
                        else:
                            if not s.perpendicular:
                                s.rotate(game_rotation+90)
                                s.perpendicular = True
                            else:
                                s.rotate(90)
                        
        
    if cv.waitKey(1) == ord('q'):
        running=False
    
    # Update

 
    fitxes.update()

    # Draw/Render
    screen.fill([17,119,17])
    fitxes.draw(screen)
    

    # [openCV] Cridem a la funcio del modul visio
    # TODO
    #
    
    # [openCV] Mostrem el joc 
    # TODO
    #
        

    # [pygame] Desem el frame de la partida
    pg.image.save(screen,'lastFrame.jpg')
    

    
    # [pygame] Actualitzem la pantalla
    pg.display.update()
pg.quit()
cv.destroyAllWindows()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
3.1415
6.283
9.4245
12.566
15.707500000000001
18.849
21.9905
25.132
28.273500000000002
31.415000000000003
34.5565
37.698
40.8395
43.981
47.1225
50.264
53.4055
56.547000000000004
59.688500000000005
62.830000000000005
65.9715
69.113
72.2545
75.39599999999999
78.53749999999998
81.67899999999997
84.82049999999997
87.96199999999996
91.10349999999995
94.24499999999995
97.38649999999994
100.52799999999993
103.66949999999993
106.81099999999992
109.95249999999992
113.09399999999991
116.2354999999999
119.3769999999999
122.51849999999989
125.65999999999988
128.8014999999999
131.9429999999999
135.0844999999999
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
Error!
138.2259999999999
141.36749999999992
144.50899999999993
147.65049999999994
150.79199999999994
153.93349999999995
157.07499999999996
160.21649999999997
163.35799999999998
166.49949999999998
169.641


hola


In [1]:
import pygame,sys
import math
from pygame.locals import *
from random import randint

## COLORS ##
white =(255,255,255)
black=(0,0,0)
red = (255,0,0)
green = (0,255,0)
greenTable=(17,119,17)
blue = (0,0,255)
orange = (255,153,0)

## WINDOW ##
heightW = 800
widthW = 1600
widthColumn = 600
heightRow = heightW/2

## TABLE ##
tableSide = 400 ## mm
laterals = 50 ##mm
areaList = []

## BASE
baseX= 1100
baseY= 600
baseHeight = 100 ##mm TODO ajustar
baseRadius = 50 ## mm

## Arm A
lenghtA = 250 ## mm
widthA = 25 ## mm
AposX = baseX+lenghtA
AposY = 400
degreeA = 0
maxDegreeA = 180

## Arm B
lenghtB = 250 ## mm
widthB = 25 ## mm
BposX = AposX+lenghtB
BposY = 400
degreeB = 90
maxDegreeB = 360 ## TODO ajustar

## Arm C
lengthC = 152 ##mm
maxDegreeC = 180
radiusC = 20 ##mm

autoRun = True



## FUNCTIONS ##

def drawTable():
    pygame.draw.rect(pWindow,greenTable,(baseX-round(tableSide/2),baseY-(tableSide + baseRadius), tableSide, tableSide))
    pygame.draw.rect(pWindow,orange,(baseX-round(tableSide/2)-laterals,baseY-(tableSide +baseRadius), laterals,tableSide))
    pygame.draw.rect(pWindow,orange,(baseX+round(tableSide/2),baseY-(tableSide +baseRadius), laterals,tableSide))

def calculateFirstArm():
    x = lenghtA * math.cos(math.radians(degreeA)) + baseX
    y = lenghtA * math.sin(math.radians(degreeA+180)) + baseY
    return x,y

def calculateSecondArm():
    x = lenghtB * math.cos(math.radians(degreeA - degreeB + 180))  + AposX
    y = lenghtB * math.sin(math.radians(degreeA - degreeB + 360)) + AposY
    return x,y

def printGrid():
    pWindow.fill(white)
    pygame.draw.line(pWindow, black, (widthColumn,0), (widthColumn, heightW))
    pygame.draw.line(pWindow, black, (0,heightRow), (widthColumn, heightRow))

def printFirstCell():
    pygame.draw.line(pWindow, blue, (0, 100),(400-(lenghtB*2),100), widthB)
    pygame.draw.line(pWindow, green, (400-(lenghtB*2),100), (400,100), widthB)

def printSecondCell():
    pygame.draw.line(pWindow, blue, (0, heightRow+(heightRow/2)),(400-(lenghtB*2),heightRow+(heightRow/2)), widthB)
    pygame.draw.line(pWindow, green, (400-(lenghtB*2),heightRow+(heightRow/2)), (400,heightRow+(heightRow/2)), widthB)

def calculateToolPath():
    pass
    ## 2*PI*radius ?


## MAIN ##

pygame.init()
pWindow= pygame.display.set_mode((widthW,heightW), pygame.RESIZABLE)
pygame.display.set_caption("R2-MINO")



while True:
    printGrid()
    drawTable()
    pygame.draw.circle(pWindow, red, (baseX,baseY), baseRadius)
    
    (AposX,AposY) = calculateFirstArm()
    pygame.draw.line(pWindow, blue, (baseX,baseY), (round(AposX),round(AposY)), widthA)

    (BposX,BposY) = calculateSecondArm()
    pygame.draw.line(pWindow, green, (round(AposX),round(AposY)), (round(BposX),round(BposY)), widthB)

    printFirstCell()
    pygame.draw.line(pWindow, orange, (350,50), (350,50+lengthC), widthB)

    printSecondCell()
    
    for pEvent in pygame.event.get():
        if pEvent.type == QUIT:
            pygame.quit()
            sys.exit()
        ## KEY controls
        elif pEvent.type == pygame.KEYDOWN:
            if pEvent.key == K_LEFT:
                if degreeA<maxDegreeA:
                    degreeA+=1
            elif pEvent.key == K_RIGHT:
                if degreeA>0:
                   degreeA-=1
            if pEvent.key == K_UP:
                if degreeB<maxDegreeB:
                    degreeB+=1
            elif pEvent.key == K_DOWN:
                if degreeB>0:
                    degreeB-=1
        ## END KEY controls

    if(autoRun == True):
        degreeB+=1
        if degreeB > maxDegreeB:
            degreeB = 0
            degreeA += 1
            if degreeA > maxDegreeA:
                degreeA = 0
        
    pygame.display.update()


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\XCoret\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
